Trying the sheets API


In [15]:
import requests
import logging
import csv
import os
import json
from datetime import datetime
from config import SPREADSHEET_ID, SHEET_NAME, API_KEY, LOCAL_CSV_PATH

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_sheet_data():
    """Fetch data from Google Sheets API"""
    url = f'https://sheets.googleapis.com/v4/spreadsheets/{SPREADSHEET_ID}/values/{SHEET_NAME}!A1:Z?alt=json&key={API_KEY}'
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if 'values' not in data:
            logger.warning("No data found in the sheet.")
            return []
        
        headers = data['values'][0]  # First row as headers
        rows = data['values'][1:]  # Remaining rows as data

        entries = [dict(zip(headers, row + [''] * (len(headers) - len(row)))) for row in rows]
        return entries

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching data from Google Sheets: {e}")
        return []

def split_vehicle_type(vehicle_type):
    """Split Vehicle Type into Cost and Vehicle Type"""
    # Example: "RMC TRUCK 250" -> Cost = "250", Vehicle Type = "RMC TRUCK"
    parts = vehicle_type.split()
    if parts and parts[-1].isdigit():  # Check if the last part is a number
        cost = parts[-1]
        vehicle = ' '.join(parts[:-1])  # Join all parts except the last one
        return cost, vehicle
    return '', vehicle_type  # If no cost is found, return empty cost and original vehicle type

def save_to_csv(entries):
    """Save formatted Google Sheets data to CSV"""
    if not entries:
        print("No data to save.")
        return

    # Define field names (headers from the first entry)
    fieldnames = list(entries[0].keys())

    # Remove 'Timestamp' and 'Vehicle Type', and add 'Date', 'Time', 'Cost', and 'Vehicle Type'
    if 'Timestamp' in fieldnames:
        fieldnames.remove('Timestamp')
    if 'Vehicle Type' in fieldnames:
        fieldnames.remove('Vehicle Type')
    fieldnames.extend(['Date', 'Time', 'Cost', 'Vehicle Type'])

    # Check if file exists to avoid rewriting headers
    file_exists = os.path.exists(LOCAL_CSV_PATH)

    # Process entries to split Timestamp, Vehicle Type, and capitalize Yes/No answers
    processed_entries = []
    for entry in entries:
        # Split the Timestamp into Date and Time
        timestamp = entry.pop('Timestamp', '')
        try:
            dt = datetime.strptime(timestamp, '%m/%d/%Y %H:%M:%S')  # Adjust format to match your data
            entry['Date'] = dt.strftime('%Y-%m-%d')
            entry['Time'] = dt.strftime('%H:%M:%S')
        except ValueError:
            entry['Date'] = ''
            entry['Time'] = ''

        # Split Vehicle Type into Cost and Vehicle Type
        vehicle_type = entry.pop('Vehicle Type', '')
        cost, vehicle = split_vehicle_type(vehicle_type)
        entry['Cost'] = cost
        entry['Vehicle Type'] = vehicle

        # Capitalize 'Yes' or 'No' answers
        for key, value in entry.items():
            if isinstance(value, str) and value.lower() in ['yes', 'no', 'y', 'n']:
                entry[key] = value.upper()

        processed_entries.append(entry)

    with open(LOCAL_CSV_PATH, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()  # Write header only once

        # Write data rows
        writer.writerows(processed_entries)

    print(f"Saved {len(processed_entries)} rows to {LOCAL_CSV_PATH}")

if __name__ == "__main__":
    entries = get_sheet_data()
    save_to_csv(entries)
    print(json.dumps(entries, indent=4))  # Print JSON for debugging

Saved 2 rows to ./processed_entries.csv
[
    {
        "1st entry or 2nd entry": "1st",
        "Material": "Sand",
        "Party Ref:": "Ozone city ",
        "Gross or Tare": "Gross",
        "Save Bill": "Y",
        "Print": "Y",
        "Date": "2025-03-09",
        "Time": "21:56:24",
        "Cost": "250",
        "Vehicle Type": "Dumper"
    },
    {
        "1st entry or 2nd entry": "1st",
        "Material": "Sand",
        "Party Ref:": "",
        "Gross or Tare": "Gross",
        "Save Bill": "Y",
        "Print": "Y",
        "Date": "2025-03-14",
        "Time": "10:21:23",
        "Cost": "250",
        "Vehicle Type": "Truck"
    }
]


Trying the email scraping

In [ ]:
import imaplib
import email
import csv
from config import USER_EMAIL, USER_PASSWORD, FROM_EMAIL

# Email credentials
EMAIL = USER_EMAIL
PASSWORD = USER_PASSWORD
IMAP_SERVER = "imap.gmail.com" 

import imaplib

def connect_to_gmail_imap(user, password):
    imap_url = 'imap.gmail.com'

    mail = imaplib.IMAP4_SSL(imap_url)
    mail.login(user, password)
    mail.select('inbox')  # Connect to the inbox.
    return mail

mail = connect_to_gmail_imap(EMAIL, PASSWORD)
status, email_ids = mail.search(None, '(FROM "{FROM_EMAIL}")')
email_ids = email_ids[0].split()

email_ids

error: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'

In [3]:

import config
print(dir(config))


['API_KEY', 'AUTOMATION_LOG', 'BOT_LOG', 'FROM_EMAIL', 'LOCAL_CSV_PATH', 'LOG_DIR', 'SHEET_NAME', 'SPREADSHEET_ID', 'USER_EMAIL', 'USER_PASSWORD', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


Scraping with .eml file

In [9]:
import email
import re
from email import policy
from email.parser import BytesParser

def extract_data_from_eml(file_path):
    """Extract structured data from a .eml file."""
    
    # Open and parse the email file
    with open(file_path, "rb") as f:
        msg = BytesParser(policy=policy.default).parse(f)
    
    # Get the email body
    if msg.is_multipart():
        body = "".join(part.get_payload(decode=True).decode(errors="ignore") for part in msg.iter_parts() if part.get_content_type() == "text/plain")
    else:
        body = msg.get_payload(decode=True).decode(errors="ignore")

    # Regex pattern to extract required fields
    pattern = re.compile(
        r"S\.N:\s*(\d+).*?"
        r"(\d{2}\.\d{2}\.\d{4},\d{2}:\d{2}).*?"
        r"V\.N:\s*([A-Z0-9]+).*?"
        r"V\.T:\s*([A-Z]+).*?"
        r"PRTY:\s*(.*?)\n.*?"
        r"MATR:\s*([A-Z\s]+).*?"
        r"CHG1:\s*(\d+).*?"
        r"G/W:\s*(\d+).*?"
        r"T/W:\s*(\d+).*?"
        r"N/W:\s*(\d+)",
        re.DOTALL
    )

    # Extract data
    match = pattern.search(body)
    if match:
        extracted_data = {
            "S.N": match.group(1),
            "Date & Time": match.group(2),
            "Vehicle Number (V.N)": match.group(3),
            "Vehicle Type (V.T)": match.group(4),
            "Party (PRTY)": match.group(5).strip() if match.group(5).strip() else "N/A",
            "Material (MATR)": match.group(6).strip(),
            "Charge (CHG1)": match.group(7),
            "Gross Weight (G/W)": match.group(8),
            "Tare Weight (T/W)": match.group(9),
            "Net Weight (N/W)": match.group(10),
        }
        return extracted_data
    else:
        return None

# Example Usage
file_path = "email.eml"  # Change this to your .eml file path
data = extract_data_from_eml(file_path)

if data:
    print("Extracted Data:")
    for key, value in data.items():
        print(f"{key}: {value}")
else:
    print("No matching data found in the email.")


Extracted Data:
S.N: 004364
Date & Time: 13.03.2025,07:58
Vehicle Number (V.N): UPCT9086
Vehicle Type (V.T): TRUCK
Party (PRTY): N/A
Material (MATR): COARSE SAND
Charge (CHG1): 150
Gross Weight (G/W): 37510
Tare Weight (T/W): 11360
Net Weight (N/W): 26150


In [13]:
!cd ..
!dir


 Volume in drive C has no label.
 Volume Serial Number is BAF7-9770

 Directory of c:\Users\Anurag Yadav\Documents\Projects\AutoK

13/03/2025  11:14 AM    <DIR>          .
10/03/2025  10:24 PM    <DIR>          ..
13/03/2025  11:14 AM                22 .gitignore
12/03/2025  10:05 PM               547 config.py
13/03/2025  11:10 AM             5,620 email.eml
12/03/2025  09:50 PM                 0 emailscraper.py
09/03/2025  10:05 PM             1,785 keyboard_sequence.py
09/03/2025  08:16 PM             7,169 LICENSE
09/03/2025  10:05 PM               103 processed_entries.csv
10/03/2025  10:25 PM               746 readme.md
10/03/2025  10:24 PM             2,029 sheets_api.py
10/03/2025  10:19 PM                 0 workings.ipynb
12/03/2025  10:06 PM    <DIR>          __pycache__
              10 File(s)         18,021 bytes
               3 Dir(s)  430,697,472,000 bytes free
